## Imports

In [1]:
%load_ext autoreload
%autoreload 2

## Oversample Data

In [90]:
import pandas as pd

df = pd.read_csv("data/raw/HIV_train.csv")
df.index = df["index"]
df["HIV_active"].value_counts()
start_index = df.iloc[0]["index"]

df.head()

,index,smiles,activity,HIV_active
index,,,,
3999,3999,CC1c2nc(N)nc(N)c2CN1C(=O)c1ccccc1,CI,0
4000,4000,Cc1nc(N)c2c(n1)C(C)N(C(=O)c1ccccc1)C2,CI,0
4001,4001,NC(=S)NN=Cc1ccc(O)cn1,CI,0
4002,4002,COC1C(OC(=O)c2ccc(C)[nH]2)C(O)C(Oc2ccc3c(O)c(N...,CM,1
4003,4003,O=C1C=C2C=CC3CC2(O1)C1CCCCN31,CI,0


In [92]:
# Check how many additional samples we need
neg_class = df["HIV_active"].value_counts()[0]
pos_class = df["HIV_active"].value_counts()[1]
multiplier = int(neg_class / pos_class) - 1

# Replicate the dataset for the positive class
replicated_pos = [df[df["HIV_active"] == 1]] * multiplier

# Append replicated data
df = pd.concat([df] + replicated_pos, ignore_index=True)

# Shuffle dataset
df = df.sample(frac=1).reset_index(drop=True)

# Re-assign index (This is our ID later)
index = range(start_index, start_index + df.shape[0])
df.index = index
df["index"] = df.index
df.head()

,index,smiles,activity,HIV_active
3999,3999,c1ccc(Nc2nnc(CCCCCCCCc3nnc(Nc4ccccc4)o3)o2)cc1,CI,0
4000,4000,O=C(C=Cc1ccc(O)c(O)c1)OC1CCCCC1OC(=O)C=Cc1ccc(...,CI,0
4001,4001,CCOC12CC(OC)C3(O)CC(C1C3OCc1ccccc1)C13C(OC)CC(...,CM,1
4002,4002,CCOC(=O)CCc1cn2cc(Br)nc(OC)c2n1,CI,0
4003,4003,Oc1nc2[nH]c(-c3ccc(Br)cc3)cc2c2ccccc12,CI,0


In [ ]:
# df.to_csv("data/raw/HIV_train_oversampled.csv", index=False)

## Training

In [7]:
from project.config import TrainingConfig
from project.train import train_model

import warnings
warnings.filterwarnings(action="ignore")

In [ ]:
config = TrainingConfig()

train_model(config, n_epochs=100)

2023-07-23 21:04:56.491 | INFO     | project.train:train_model:129 - Loading the dataset...
2023-07-23 21:04:56.714 | INFO     | project.train:train_model:146 - Constructed model with 365633 parameters
100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 560/560 [00:31<00:00, 17.92it/s]
2023-07-23 21:05:28.029 | INFO     | project.train:train_model:168 - Epoch 0 | Train Loss 0.784846593439579
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:01<00:00, 19.63it/s]
2023-07-23 21:05:29.679 | INFO     | project.train:train_model:176 - Epoch 0 | Test Loss 0.725645499303937
100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 560/560 [00:31<00:00, 17.90it/s]
2023-07-23 21:06:03.737 | INFO     | project.train:train_model:168 - Epoch 1 | Train Loss 0.7848186417349747
100%|████████████████████████████████████████████████████